# GET all "gurupearls and stupas" products

In [1]:
from lxml.etree import HTML
from re import search, M, I
from requests import get, Session
from itertools import chain
from json import dump, load

In [2]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'
session = Session()
session.headers.update({
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'en-US;q=0.6,en;q=0.4',
    'Connection': 'keep-alive',
    'Content-Length': '0',
    'Host': 'www.malawerkstatt.de',
    'Origin': 'https://www.malawerkstatt.de/',
    'Referer': 'https://www.malawerkstatt.de/',
    'User-Agent': user_agent
})

In [3]:
base_url = 'https://www.malawerkstatt.de'
page_url = 'https://www.malawerkstatt.de/epages/61255242.sf/de_DE/'
url = 'https://www.malawerkstatt.de/epages/61255242.sf/de_DE/?ViewAction=View&ObjectID=53284986&PageSize=60&Page=%s'

In [4]:
number_of_pages = 3
pages = [session.get(url % n).text for n in range(1,number_of_pages+1)]
trees = [HTML(page) for page in pages]

In [5]:
size_re = r'\d*(\.\d|\,\d|).?mm'

In [6]:
def get_size_from_text(text):
    ''' returns size as string from string '''
    try:
        return search(size_re, text, M|I).group().strip()
    except Exception as e:
        print 'Error in get_size_from_text: %s' % e
        return ''
    
def get_text_from_tree(tree, x_path):
    ''' returns description as string '''
    try:
        description_fragments = list(tree.xpath(x_path).pop().itertext())
        return ' '.join([fragment.strip() 
                         for fragment in description_fragments if fragment.strip()])
    except Exception as e:
        print 'Error in get_text_from_tree: %s' % e
        return ''
    
def text_to_float(text_with_num):
    ''' returns float from text with number in it '''
    if text_with_num:
        return float(search(r'\d+(.?\d+|)', text_with_num, M|I).group().replace(',','.'))
    else:
        return None

def get_product_details(tree):
    ''' takes product tree and returns dict with details '''
    title = tree.xpath('.//h3').pop()
    title_text = get_text_from_tree(tree=title, x_path='.//a')
    size = get_size_from_text(title_text)
    price = get_text_from_tree(tree, x_path='.//span[@class="Price"]')
    return dict(
        img_url=base_url + tree.xpath('.//img[@class="ProductSmallImage"]/@src').pop(),
        title=title_text,
        description=get_text_from_tree(tree, x_path='.//div[@class="Description"]'),
        url=page_url + title.xpath('.//a/@href').pop(),
        size_num= text_to_float(size),
        price_num= text_to_float(price),
        size=size,
        price=price
    )

def get_products_from_page(tree):
    ''' returns list with all product entries as trees '''
    products_wrapper = tree.xpath('.//div[@id="CategoryProducts"]').pop()
    return products_wrapper.xpath('.//div[@class="InfoArea"]')

In [7]:
products_by_page = [
    [get_product_details(product) for product in get_products_from_page(page)] 
        for page in trees]

Error in get_size_from_text: 'NoneType' object has no attribute 'group'
Error in get_size_from_text: 'NoneType' object has no attribute 'group'
Error in get_size_from_text: 'NoneType' object has no attribute 'group'
Error in get_size_from_text: 'NoneType' object has no attribute 'group'
Error in get_size_from_text: 'NoneType' object has no attribute 'group'
Error in get_size_from_text: 'NoneType' object has no attribute 'group'
Error in get_size_from_text: 'NoneType' object has no attribute 'group'
Error in get_size_from_text: 'NoneType' object has no attribute 'group'
Error in get_size_from_text: 'NoneType' object has no attribute 'group'
Error in get_size_from_text: 'NoneType' object has no attribute 'group'


In [8]:
all_products = list(chain.from_iterable(products_by_page))

## Save to file

In [9]:
dump(all_products, open('guruperlen_stupa_products.min.json','w'), encoding='utf-8')